In [22]:
import numpy as np
import os
import sys

sys.path.append(os.path.join(os.getenv("HOME"), "gpuaffman_networks/"))
import general_network, ragged_general_network
np.random.seed(11321)

The state of an N node boolean network is specified by a vector of N binary variables

In [17]:
N = 8
state = np.random.binomial(1, 0.5, N).astype(np.bool_)
print(state)

[ True False False  True False  True False False]


In a ragged network, every node is wired to at most $k_{max}$ other nodes. We specify this connectivity via an $N \times k_{max}$ integer matrix and an $N \times k_{max}$ boolean matrix. The boolean matrix masks the integer matrix, and connections are active when the boolean matrix is True


In [18]:
k_max = 3
connectivity = np.random.randint(0, N, (N, k_max)).astype(np.uint8)
used_connectivity = np.random.binomial(1, 0.5, (N, k_max)).astype(np.bool_)
print(connectivity)
print(used_connectivity)


[[6 4 2]
 [1 0 3]
 [3 1 4]
 [2 6 0]
 [6 4 6]
 [2 3 1]
 [0 3 1]
 [6 5 0]]
[[ True  True False]
 [ True  True False]
 [ True False  True]
 [ True  True False]
 [ True  True False]
 [False  True  True]
 [ True  True  True]
 [False  True False]]


In this example, the active connections associated with node 0 are 6 and 4. The connection between node 0 and 2 is not active.


At each timestep, the state of a given node is updated according to the states of all the nodes it is connected to. This update rule is specified by a truth table for each node, an $N \times 2^{k_{max}}$ boolean matrix


In [20]:
functions = np.random.binomial(1, 0.5, (N, 1 << k_max)).astype(np.bool_)
print(functions)


[[False False  True  True  True False False False]
 [False False  True  True False  True  True  True]
 [False  True  True  True  True  True False  True]
 [False False  True False  True  True  True False]
 [False  True  True  True False False  True False]
 [ True  True False False  True False False False]
 [ True  True False False  True  True False False]
 [ True False  True False False  True False False]]


So in this example, if at time $t$ nodes 6 and 4 are both in the state False, at time $t+1$ the state of node 0 will be updated to 0.


The functions in ragged_general_network.py do batch updating of the states of boolean networks. The inputs are the state at time $t$, the connectivity matrices, and the matrix of functions. The output is the state at time $t+1$.


The first step is to construct an $N \times k_{max}$ matrix containing the state values used to update each node. I accomplish this with matrix slicing in general_network.get_update_inputs


In [23]:
update_states = general_network.get_update_inputs(state, connectivity)
print(update_states)


[[False False False]
 [False  True  True]
 [ True False False]
 [False False  True]
 [False False False]
 [False  True False]
 [ True  True False]
 [False  True  True]]


We can see that each row of the matrix contains the states at positions specified by connectivity. For example, the first row contains the states of nodes 6, 4 and 2.


Next I convert each row of update_states into it's corresponding integer value, taking into account the masking of used_connectivity. This is done using binary_core.binary_to_uint8. I then use the produced values to slice functions, which results in a vector containing the state at time $t+1$. This is accomplished by ragged_general_network.apply_ragged_k_binary_function


In [25]:
state_tp1 = ragged_general_network.apply_ragged_k_binary_function(update_states, functions, used_connectivity)
print(state_tp1)


[False  True  True False False False False  True]
